In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import gc

In [2]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
df = pd.read_csv(r"C:\Users\krish\Downloads\archive (8)\NIFTY50_all.csv")

# Drop non-numeric columns
df.drop(columns=["Symbol", "Series"], inplace=True)

for col in df.columns:
    if df[col].dtype == "object":  
        df[col].fillna(df[col].mode()[0], inplace=True)  # Fill categorical with most frequent
    else:
        df[col].fillna(df[col].median(), inplace=True)  # Fill numeric with median
        
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df.iloc[:, 1:]), columns=df.columns[1:])  # Skip date column


C:\Users\krish\AppData\Local\Temp\ipykernel_7152\1122546878.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)  # Fill categorical with most frequent
C:\Users\krish\AppData\Local\Temp\ipykernel_7152\1122546878.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting valu

In [4]:
SEQ_LENGTH = 100

In [5]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length].values)  # Past 30 days
        y.append(data.iloc[i + seq_length].values)  # Predict next day
    return np.array(X), np.array(y)

In [6]:
X, y = create_sequences(df_scaled, SEQ_LENGTH)
X_train, y_train = torch.tensor(X[:-200], dtype=torch.float32), torch.tensor(y[:-200], dtype=torch.float32)
X_test, y_test = torch.tensor(X[-200:], dtype=torch.float32), torch.tensor(y[-200:], dtype=torch.float32)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

In [8]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [9]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, output_dim=10):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :])

In [10]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, output_dim=10):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

In [11]:
class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, output_dim=10):
        super(BiLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Double size due to bidirection

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

In [12]:
class LSTMWithDropout(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=2, output_dim=10, dropout=0.3):
        super(LSTMWithDropout, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

In [13]:
input_dim = X_train.shape[2]
output_dim = y_train.shape[1]

models = {
    "RNN": RNNModel(input_dim, output_dim=output_dim).to(device),
    "LSTM": LSTMModel(input_dim, output_dim=output_dim).to(device),
    "BiLSTM": BiLSTMModel(input_dim, output_dim=output_dim).to(device),
    "LSTM_Dropout": LSTMWithDropout(input_dim, output_dim=output_dim).to(device),
}


In [14]:
def train_model(model, name, epochs=10):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f" {name} - Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")


In [15]:
for name, model in models.items():
    print(f" Training {name} model...")
    train_model(model, name)

 Training RNN model...
 RNN - Epoch 1/10, Loss: 0.0011
 RNN - Epoch 2/10, Loss: 0.0023
 RNN - Epoch 3/10, Loss: 0.0012
 RNN - Epoch 4/10, Loss: 0.0011
 RNN - Epoch 5/10, Loss: 0.0011
 RNN - Epoch 6/10, Loss: 0.0011
 RNN - Epoch 7/10, Loss: 0.0011
 RNN - Epoch 8/10, Loss: 0.0011
 RNN - Epoch 9/10, Loss: 0.0011
 RNN - Epoch 10/10, Loss: 0.0011
 Training LSTM model...
 LSTM - Epoch 1/10, Loss: 0.0011
 LSTM - Epoch 2/10, Loss: 0.0010
 LSTM - Epoch 3/10, Loss: 0.0010
 LSTM - Epoch 4/10, Loss: 0.0010
 LSTM - Epoch 5/10, Loss: 0.0010
 LSTM - Epoch 6/10, Loss: 0.0010
 LSTM - Epoch 7/10, Loss: 0.0010
 LSTM - Epoch 8/10, Loss: 0.0010
 LSTM - Epoch 9/10, Loss: 0.0010
 LSTM - Epoch 10/10, Loss: 0.0010
 Training BiLSTM model...
 BiLSTM - Epoch 1/10, Loss: 0.0010
 BiLSTM - Epoch 2/10, Loss: 0.0010
 BiLSTM - Epoch 3/10, Loss: 0.0010
 BiLSTM - Epoch 4/10, Loss: 0.0010
 BiLSTM - Epoch 5/10, Loss: 0.0010
 BiLSTM - Epoch 6/10, Loss: 0.0010
 BiLSTM - Epoch 7/10, Loss: 0.0010
 BiLSTM - Epoch 8/10, Loss: 0.

In [16]:
def evaluate_model(model, X_test, y_test, name):
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)
        mse = nn.MSELoss()(y_pred, y_test).item()
    print(f"{name} MSE: {mse:.6f}")

In [17]:
for name, model in models.items():
    evaluate_model(model, X_test, y_test, name)

RNN MSE: 0.000611
LSTM MSE: 0.000549
BiLSTM MSE: 0.000544
LSTM_Dropout MSE: 0.000543
